In [ ]:
import pandas as pd
import numpy as np
from pandas import melt

https://office.wikimedia.org/wiki/Quarters

In [ ]:
all_funded = pd.read_csv('inputs/data-all-funded.csv')

In [ ]:
# clean all_funded

#drop rows without USD amount
df = all_funded[all_funded['USD over grant life'] != 0]
df = df[df['USD over grant life'].notnull()]

#fix datatype for USD over grant life column
df['USD over grant life'] = df['USD over grant life'].str.replace(',', '')
df['USD over grant life'] = df['USD over grant life'].str.replace('$', '')
df['USD over grant life'] = df['USD over grant life'].astype('float')

#columns to datetime
df['Approved on'] =  pd.to_datetime(df['Approved on'], errors = 'coerce')
df['Executed on'] =  pd.to_datetime(df['Executed on'], errors = 'coerce')

df['grant_count'] = df.groupby('Grantee').cumcount() + 1
df["total_grantee_grants"] = df.groupby('Grantee') ['Approved on'].transform('count')+1 #to start the count at 

df['counter'] = range(len(df))

#assign unique IDs
df['id'] = df.groupby('Grantee').ngroup()

In [ ]:
df['Program'].unique()
#grants_df = df[df['grant_count'] != 'Conference']

#do not include TPS, PEG, IEG, Partnership Grants
exclude = ['TPS', 'PEG', 'IEG', 'Partnership Grants', 'Conference', 'Wikicite', 'WMS']

grants_df = df[~df['Program'].isin(exclude)]

In [ ]:
grants_df20 = grants_df[grants_df['Fiscal year ending'] == 2020]

### KR 1: Ensure 65% of all grants are from outside well established communities so that grantmaking becomes a key mechanism to empower and welcome newcomers and increase diversity of content.

In [ ]:
grants_by_community = grants_df20.groupby(['Fiscal year ending', 'Community type'])['counter'].nunique().to_frame().rename(columns={'counter': 'unique_grants'}).reset_index()
grants_by_community[grants_by_community['Community type']!='Developed']['unique_grants'].sum()/len(grants_df20)

### % of grantees who are new*

In [ ]:
total_grants = grants_df20.groupby('Fiscal year ending').size().to_frame().reset_index().rename(columns={0: 'total_granted'})
unique_grantees = grants_df20.groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'unique_grantees'})
unique_new_grantees = grants_df20[grants_df20['grant_count'] == 1].groupby('Fiscal year ending')['Grantee'].nunique().to_frame().reset_index().rename(columns={'Grantee': 'unique_n_grantees'})

#get a count of all grants awarded to a new grantee that has not received a grant in a previous year, not including those grantees that doubled up in their first
#year and received a follow-up grant
new_grantee_grants = pd.pivot_table(data=grants_df20[grants_df20['grant_count'] == 1], index='Fiscal year ending', values='counter', aggfunc='count').reset_index().rename(columns={'counter': 'total_n_grantee_grants'})

#create roll_up df combining the dfs above
year_roll_up = total_grants.merge(unique_grantees, on='Fiscal year ending', how='left').merge(unique_new_grantees, on='Fiscal year ending', how='left')
year_roll_up['n_grantee %'] = year_roll_up['unique_n_grantees'] / year_roll_up['unique_grantees']

In [ ]:
year_roll_up

### % of grantees who are new from emerging community*

In [ ]:
unique_new_grantees_by_comm = grants_df20[grants_df20['grant_count'] == 1].groupby(['Fiscal year ending', 'Community type'])['Grantee'].nunique().to_frame().rename(columns={'Grantee': 'unique_n_grantees'}).reset_index()
unique_new_grantees_by_comm[unique_new_grantees_by_comm['Community type'] == 'Emerging']['unique_n_grantees'].sum()/unique_new_grantees_by_comm['unique_n_grantees'].sum()

### % of all funds for all grants in emerging and least developed communities

In [ ]:
grants_df20[grants_df20['Community type'].str.match('Emerging|Least Developed')]['USD over grant life'].sum()/grants_df20['USD over grant life'].sum()

### % gender focused grant funding

In [ ]:
grants_df20[grants_df20['Gender gap (Y/N)'] == 'Yes']['USD over grant life'].sum()/grants_df20['USD over grant life'].sum()

### % of rapid grantees that had more than one rapid grant in the reporting year

In [ ]:
rgrbg_20 = grants_df20[grants_df20['Program'] == 'Rapid'].groupby('Grantee').size().to_frame().reset_index().rename(columns={0: 'rapid_grants_received'})
rgrbg_20_receiving_multiple = len(rgrbg_20[rgrbg_20['rapid_grants_received'] >= 2])/len(rgrbg_20)
rgrbg_20_receiving_multiple

### % of rapid grant grantees that received a rapid grant in the year prior

In [ ]:
rapid_20 = df[(df['Program']=='Rapid') & (df['Fiscal year ending']==2020)]
rapid_19 = df[(df['Program']=='Rapid') & (df['Fiscal year ending']==2019)]

r19list = list(rapid_19['Grantee'].unique())
r_grantees_2020_that_received_prior_r_grant  = rapid_20[rapid_20['Grantee'].isin(r19list)]

print('r_grantees_2020_that_received_prior_r_grant:', len(r_grantees_2020_that_received_prior_r_grant)) 
print('2019 rapid Grantees:', len(r19list))
print('2020 rapid Grantees:', len(rapid_20['Grantee'].unique()))
print('2020 R grant Grantees that received prior rapid grants in prior year:',(len(r_grantees_2020_that_received_prior_r_grant)/len(rapid_20['Grantee'].unique())*100), '%')